# Applied Data Science Capstone (Week 3) 

<img src="img/toronto-neigh.png"/>

<hr/>

## Introduction
In this mini project, we will explore and analyze neighborhoods in **Toronto**. 

In the first part we will request and scrape [this](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) Wikipedia page to get the neighborhoods and boroughs in Toronto, using BeautifulSoup (bs4), and store the data as pandas DataFrame. We will add geospatial data to the data using a csv file (included in this repository). We will visulize the neighborhoods in map created using folium package. Onece the data is created we will use the Foursquare API to explore neighborhoods.

The second part we will explore more three selected boroughs in Toronto. We will get all the venues nerby and store them in a new dataset.

The third part is an analysis of neighborhoods in the dataset. We will create a new dataset by encoding venue categies as dummy variables. We will also display every neighborhood with top 5 most frequent venue categories.

In the fourth part we will use K means method to cluster the neighborhoods and visualize clustors on a map.

In the fifth part we will display clusters to get an insight on what categories distinguishes what cluster.   

*NB* : To be able to exploit Foursquare API you neet to plug your credentials (Client ID and Client Secret). Please go to Foursquare [website](https://foursquare.com/developers/signup) an create an account to get your credentials. 

## Table of contents
* [1. Prepare the Dataset](#ded)
* [2. Explore Neighborhoods in Toronto](#ent)
* [3. Analyze Each Neighborhood](#aen)
* [4. Cluster Neighborhoods](#cn)
* [5. Examine Clusters](#ec)

Importing required Python libraries

In [1]:
# Packages for data analysis
import pandas as pd
import numpy as np

# Module to handle
from pandas.io.json import json_normalize

# KMean for clustoring
from sklearn.cluster import KMeans

# Packages for requesting a url and scraping 
from bs4 import BeautifulSoup
import requests

# Getting geo coordinates
from geopy.geocoders import Nominatim

# Maps
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Prepare the Dataset <a name="ded"></a>

Getting the wikipedia html page 

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html,'html.parser')

By analyzing the html code we can see that the data is held inside a table tag referenced by class = "wikitable sortable". So let's use BeautifulSoup to get the right tag

In [3]:
tab = soup.find(class_="wikitable sortable")

Creating a Pandas DataFrame to store the data

In [4]:
df_neighborhoods = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])
df_neighborhoods

,PostalCode,Borough,Neighborhood


Parsing the wiki page and filling the data <br/>
*Note: I started by checking if there is 'Not assigned' value for borough and neighborhood and I inseart or update a row*

In [5]:
for row in tab.find_all('tr'):   
    if row.th:
        pass # ignoring the header row
    else:
        row_list = row.find_all('td')
        postalcode = row_list[0].string
        borough = row_list[1].a.string if row_list[1].a else row_list[1].string
        neighborhood = row_list[2].a.string if row_list[2].a else row_list[2].string
        neighborhood = neighborhood[:len(neighborhood)-1] if neighborhood.endswith('\n') else neighborhood # getting rid of '\n'
        if borough != 'Not assigned':
            if neighborhood == 'Not assigned':
                df_neighborhoods = df_neighborhoods \
                                            .append({'PostalCode':postalcode, \
                                                    'Borough':borough, \
                                                    'Neighborhood':borough}, \
                                                    ignore_index=True)
            else:
                msk = df_neighborhoods['PostalCode'] == postalcode
                if df_neighborhoods[msk].shape[0] == 0:                
                    df_neighborhoods = df_neighborhoods \
                                            .append({'PostalCode':str(postalcode), \
                                                    'Borough':str(borough), \
                                                    'Neighborhood':str(neighborhood)}, \
                                                    ignore_index=True)
                else: # neighborhood with postal code found so we update neighborhood column only
                    df_neighborhoods.loc[msk,'Neighborhood'] += ', ' + neighborhood    

In [6]:
df_neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [7]:
df_neighborhoods.shape

(103, 3)

In [8]:
filename = 'Geospatial_Coordinates.csv'
df_coordinates = pd.read_csv(filename)
df_coordinates.rename(columns = {'Postal Code':'PostalCode'},inplace=True)
df_coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df_coordinates.shape

(103, 3)

Let's join these two datasets to associate every neighborhood with its geospatial coordinates

In [10]:
df_neighborhoods = pd.merge(df_neighborhoods,df_coordinates,on = 'PostalCode')
df_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


From now on we will keep only the rows with borough containning 'Toronto'

In [11]:
df_toronto = df_neighborhoods[df_neighborhoods['Borough'].apply(lambda x: 'Toronto' in x)].reset_index(drop=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [12]:
df_toronto['Borough'].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

In [13]:
df_toronto.shape

(38, 5)

We will visualize the neiborhoods on a maps using folium. Let's get Toronto's coordinates

In [14]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ton_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
map_toronto = folium.Map(location = [latitude,longitude], zoom_start = 12)

for lat,lng,neighborhood,borough in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Neighborhood'],df_toronto['Borough']):
    label='{},{}'.format(neighborhood,borough)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 

map_toronto

#### Define Foursquare Credentials and Version

<div class="alert alert-danger alertdanger" style="margin-top: 5px" align="center">
<h3>You need to plug your Foursquare credentials !</h3>
</div>

In [43]:
CLIENT_ID = 'your-client-id' # your Foursquare ID
CLIENT_SECRET = 'your-secret-id' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: your-client-id
CLIENT_SECRET:your-secret-id


#### Let's explore the first neighborhood in our dataframe.lets explore the first neighborhood

In [17]:
name = df_toronto.loc[0,'Neighborhood']
latitude = df_toronto.loc[0,'Latitude']
longitude = df_toronto.loc[0,'Longitude']

print('Latitude and longitude values of \"{}\" are {}, {}.'.format(name,latitude,longitude))

Latitude and longitude values of "Harbourfront, Regent Park" are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [44]:
RADIUS = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    RADIUS, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=your-client-id&client_secret=your-secret-id&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
results

{u'meta': {u'code': 200, u'requestId': u'5c9e01d51ed2192729a8037d'},
 u'response': {u'groups': [{u'items': [{u'reasons': {u'count': 0,
       u'items': [{u'reasonName': u'globalInteractionReason',
         u'summary': u'This spot is popular',
         u'type': u'general'}]},
      u'referralId': u'e-0-54ea41ad498e9a11e9e13308-0',
      u'venue': {u'categories': [{u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          u'suffix': u'.png'},
         u'id': u'4bf58dd8d48988d16a941735',
         u'name': u'Bakery',
         u'pluralName': u'Bakeries',
         u'primary': True,
         u'shortName': u'Bakery'}],
       u'id': u'54ea41ad498e9a11e9e13308',
       u'location': {u'address': u'362 King St E',
        u'cc': u'CA',
        u'city': u'Toronto',
        u'country': u'Canada',
        u'crossStreet': u'Trinity St',
        u'distance': 143,
        u'formattedAddress': [u'362 King St E (Trinity St)',
         u'Toronto ON M5A 1K9',
         u'Canada'

In [20]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']    

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

48 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto <a name="ent"></a>

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

#### Let's check the size of the resulting dataframe

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(1712, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


#### Let's find out how many unique categories can be curated from all the returned venues

In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


## 3. Analyze Each Neighborhood <a name="aen"></a>

In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [28]:
toronto_onehot.shape

(1712, 240)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.045455,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.00,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.011765,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.011765,0.000000,0.000000,0.011765,0.000000,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.050000,0.000000,0.040000,0.010000,0.000000,0.000000,0.00
8,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.023256,0.00,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.011628,0.011628,0.000000,0.000000,0.011628,0.00


#### Let's confirm the new size

In [30]:
toronto_grouped.shape

(38, 240)

#### Let's print each neighborhood along with the top 5 most common venues

In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1       Steakhouse  0.04
2             Café  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.03
3          Steakhouse  0.03
4      Farmers Market  0.03


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3   Grocery Store  0.05
4   Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0   Comic Shop  0.06
1   Restaurant  0.06
2          Spa  0.06
3  Pizza Place  0.06
4   Smoke Shop  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3  Sculpture Garden  

#### Let's put that into a *pandas* dataframe

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Thai Restaurant,Café,Steakhouse,Bar,Hotel,American Restaurant,Bakery,Sushi Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Restaurant,Café,Pub
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Bar,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store
3,Business Reply Mail Processing Centre 969 Eastern,Auto Workshop,Garden,Brewery,Farmers Market,Spa,Light Rail Station,Fast Food Restaurant,Burrito Place,Restaurant,Recording Studio
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Plane,Sculpture Garden,Boutique,Boat or Ferry,Harbor / Marina,Airport Gate,Airport


## 4. Cluster Neighborhoods <a name="cn"></a>

In [34]:
# set number of clusters
kclusters = 5
print('Run k-means to cluster the neighborhood into {} clusters.'.format(kclusters))

Run k-means to cluster the neighborhood into 5 clusters.


In [35]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Bakery,Café,Park,Pub,Theater,Breakfast Spot,Mexican Restaurant,Restaurant,Historic Site
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Pizza Place,Diner,Italian Restaurant,Japanese Restaurant,Tea Room
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Clothing Store,Gastropub,Cosmetics Shop,Bakery,Italian Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Restaurant,Café,Pub


Finally, let's visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], \
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters <a name="ec"></a>

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,0,Health Food Store,Coffee Shop,Pub,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
30,Central Toronto,0,Coffee Shop,Pub,Pizza Place,American Restaurant,Light Rail Station,Medical Center,Sports Bar,Supermarket,Sushi Restaurant,Fried Chicken Joint


#### Cluster 2

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Bakery,Café,Park,Pub,Theater,Breakfast Spot,Mexican Restaurant,Restaurant,Historic Site
1,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Pizza Place,Diner,Italian Restaurant,Japanese Restaurant,Tea Room
2,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Clothing Store,Gastropub,Cosmetics Shop,Bakery,Italian Restaurant
4,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Restaurant,Café,Pub
5,Downtown Toronto,1,Coffee Shop,Café,Italian Restaurant,Middle Eastern Restaurant,Burger Joint,Bar,Bubble Tea Shop,Japanese Restaurant,Ice Cream Shop,Thai Restaurant
6,Downtown Toronto,1,Grocery Store,Café,Park,Restaurant,Baby Store,Italian Restaurant,Athletics & Sports,Nightclub,Coffee Shop,Convenience Store
7,Downtown Toronto,1,Coffee Shop,Thai Restaurant,Café,Steakhouse,Bar,Hotel,American Restaurant,Bakery,Sushi Restaurant,Asian Restaurant
8,West Toronto,1,Pharmacy,Supermarket,Discount Store,Bakery,Liquor Store,Middle Eastern Restaurant,Music Venue,Park,Coffee Shop,Pool
9,Downtown Toronto,1,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Pizza Place,Brewery,Fried Chicken Joint,Scenic Lookout,Restaurant
10,West Toronto,1,Bar,Coffee Shop,Asian Restaurant,Café,Restaurant,Pizza Place,Boutique,Cocktail Bar,Bakery,Men's Store


#### Cluster 3

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,2,Playground,Gym,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 4

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,3,Gym / Fitness Center,Park,Swim School,Bus Line,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
32,Downtown Toronto,3,Park,Trail,Playground,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Women's Store


#### Cluster 5

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,4,Home Service,Garden,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


<hr/>

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

Thanks for completing this notebook! If you have any questions feel free to contact me on [linkedIn](https://www.linkedin.com/in/hichamghanim/) or on [Twitter](https://twitter.com/HichamGhanim)